In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import inforec
import notebook_utils as nu
from model import EventBuilder

DB_DIR = 'data'

In [ ]:
inforec.InfoRecDB.init(DB_DIR)

In [4]:
db = inforec.InfoRecDB.open(DB_DIR)

In [5]:
nu.dump_events(db)

,ID,Title,Before,After,Same
0,ca3bcf47-c503-4795-ae61-e76d2144b642,title1,None,None,None
1,495d6e53-04e9-48b3-bccb-67d03f081a28,Title2,None,None,None
2,209bd96c-74bf-4ff2-87fc-ca97a36558af,title3,[title1],None,None


In [10]:
e1 = EventBuilder("title1").build()
e2 = EventBuilder("Title2").desc("description for title2").build()
e3 = EventBuilder("title3").before(e1.id).build()
db.add_item(e1)
db.add_item(e2)
db.add_item(e3)

In [ ]:
e4 = 

In [6]:
nu.dump_events(db)

,ID,Title,Before,After,Same
0,ca3bcf47-c503-4795-ae61-e76d2144b642,title1,None,None,None
1,495d6e53-04e9-48b3-bccb-67d03f081a28,Title2,None,None,None
2,209bd96c-74bf-4ff2-87fc-ca97a36558af,title3,[title1],None,None


In [14]:
db.write()

In [6]:
event = EventBuilder("title4").before('209bd96c-74bf-4ff2-87fc-ca97a36558af').after('ca3bcf47-c503-4795-ae61-e76d2144b642').build()
db.add_item(event)
nu.dump_events(db)

,ID,Title,Before,After,Same
0,ca3bcf47-c503-4795-ae61-e76d2144b642,title1,None,None,None
1,495d6e53-04e9-48b3-bccb-67d03f081a28,Title2,None,None,None
2,209bd96c-74bf-4ff2-87fc-ca97a36558af,title3,[title1],None,None
3,a8ce1293-8748-40ad-93e4-30546331e513,title4,[title3],[title1],None


In [7]:
db.has_no_conflict() or db.conflicts() or [[ db.get_event(eid).title for eid in lst ] for lst in db.conflicts()]

True

In [1]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.
